https://www.kaggle.com/austinreese/craigslist-carstrucks-data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Display preferences
%matplotlib inline
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [25]:
# Create dataframe
cars = pd.read_csv('../../data/vehicles.csv')

### Data Cleaning 

In [3]:
print('Number of rows: {}'.format(cars.shape[0]))
print('Number of columns: {}'.format(cars.shape[1]))

Number of rows: 509577
Number of columns: 25


In [4]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509577 entries, 0 to 509576
Data columns (total 25 columns):
id              509577 non-null int64
url             509577 non-null object
region          509577 non-null object
region_url      509577 non-null object
price           509577 non-null int64
year            508050 non-null float64
manufacturer    486813 non-null object
model           501588 non-null object
condition       277643 non-null object
cylinders       309894 non-null object
fuel            505592 non-null object
odometer        417253 non-null float64
title_status    506515 non-null object
transmission    505858 non-null object
vin             302152 non-null object
drive           365434 non-null object
size            167574 non-null object
type            368046 non-null object
paint_color     344871 non-null object
image_url       509563 non-null object
description     509561 non-null object
county          0 non-null float64
state           509577 non-null obj

In [9]:
cars.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7034441763,https://saltlakecity.craigslist.org/cto/d/salt...,salt lake city,https://saltlakecity.craigslist.org,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,63500.0,clean,manual,WVWPF7AJ6CW316713,4wd,compact,hatchback,black,https://images.craigslist.org/00G0G_fTLDWM5Xyv...,PRICE REDUCED! -Garage kept -Low Miles (63K)...,NaN,ut,40.7372,-111.858
1,7034440610,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,gas,10.0,clean,automatic,1FTMF1EP3GKF13544,4wd,NaN,NaN,NaN,https://images.craigslist.org/00v0v_7Cu0buIofU...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
2,7034440588,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,46463,2015.0,gmc,sierra 1500,excellent,NaN,gas,7554.0,clean,automatic,3GTU2WEC6FG228025,4wd,NaN,NaN,white,https://images.craigslist.org/01515_lPvJ9bfbdY...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
3,7034440546,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,gas,10.0,clean,automatic,1FTEX1EF6GKD25447,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_6Rjfp3NS4O...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
4,7034406932,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,49999,2018.0,ford,f-450,NaN,NaN,diesel,70150.0,clean,automatic,1FT8W4DT8GEA90427,4wd,NaN,pickup,white,https://images.craigslist.org/00W0W_8yIUwRBXXd...,2018 Ford F-350 F350 F 350 SD Lariat Crew Cab ...,NaN,ut,40.3744,-104.694


In [10]:
cars.tail()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
509572,7034554007,https://saltlakecity.craigslist.org/ctd/d/engl...,salt lake city,https://saltlakecity.craigslist.org,15476,2012.0,nissan,xterra,NaN,6 cylinders,gas,103135.0,clean,automatic,5N1AN0NW0CC520698,4wd,NaN,SUV,NaN,https://images.craigslist.org/00z0z_3ERBmVoD5s...,Maximum Auto SearchÂ Â Â Â Â Â Â Â STOCK #:Â Â...,NaN,ut,39.6405,-104.961
509573,7034538385,https://saltlakecity.craigslist.org/ctd/d/engl...,salt lake city,https://saltlakecity.craigslist.org,9881,2008.0,bmw,3 series 328i,NaN,6 cylinders,gas,108277.0,clean,automatic,WBAWR33558P150498,rwd,NaN,convertible,NaN,https://images.craigslist.org/01313_38rMiNAW18...,Maximum Auto SearchÂ Â Â Â Â Â Â Â STOCK #:Â Â...,NaN,ut,39.6405,-104.961
509574,7034523643,https://saltlakecity.craigslist.org/ctd/d/salm...,salt lake city,https://saltlakecity.craigslist.org,24895,2019.0,jeep,cherokee,NaN,4 cylinders,gas,6973.0,clean,automatic,1C4PJMLB0KD267506,4wd,NaN,SUV,white,https://images.craigslist.org/00606_dOtGS5RmTi...,"2019 Jeep Cherokee Latitude Plus -- $24,895 ...",NaN,ut,45.1637,-113.894
509575,7034458858,https://saltlakecity.craigslist.org/cto/d/ogde...,salt lake city,https://saltlakecity.craigslist.org,32500,2017.0,NaN,Porsche Macan GTS,excellent,6 cylinders,gas,37520.0,clean,automatic,NaN,NaN,NaN,NaN,red,https://images.craigslist.org/00s0s_32yAwr0KMt...,2017 Porsche Macan GTS Carmine Red w/ Black ...,NaN,ut,41.3112,-111.969
509576,7034389294,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,12900,2012.0,ram,NaN,NaN,NaN,gas,152296.0,clean,automatic,3C6TD5HT1CG309173,4wd,NaN,pickup,black,https://images.craigslist.org/00E0E_3FtnUA2XPT...,"2012 RAM 2500 4WD Crew Cab 169"" ST"" Offere...",NaN,ut,40.3744,-104.694


In [13]:
cars.describe()

,id,price,year,odometer,county,lat,long
count,5.095770e+05,5.095770e+05,508050.000000,4.172530e+05,0.0,499285.000000,499285.000000
mean,7.044176e+09,5.479684e+04,2009.662236,1.017300e+05,NaN,38.453818,-94.313886
std,4.937219e+06,9.575025e+06,8.567953,1.073790e+05,NaN,5.902152,17.717653
min,6.995212e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-82.688100,-165.935000
25%,7.040802e+09,3.995000e+03,2007.000000,4.948800e+04,NaN,34.557400,-108.506000
50%,7.045325e+09,9.377000e+03,2011.000000,9.489400e+04,NaN,39.145300,-88.678800
75%,7.048556e+09,1.795500e+04,2015.000000,1.387780e+05,NaN,42.449000,-81.111400
max,7.050103e+09,3.600029e+09,2021.000000,1.000000e+07,NaN,81.569300,94.124800


In [14]:
cars.describe(include=['O'])

,url,region,region_url,manufacturer,model,condition,cylinders,fuel,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,state
count,509577,509577,509577,486813,501588,277643,309894,505592,506515,505858,302152,365434,167574,368046,344871,509563,509561,509577
unique,509577,403,413,43,35852,6,8,5,6,3,180145,3,4,13,12,349468,427803,51
top,https://butte.craigslist.org/ctd/d/butte-2005-...,fayetteville,https://modesto.craigslist.org,ford,f-150,excellent,6 cylinders,gas,clean,automatic,WDC0G4JB4HF164180,4wd,full-size,sedan,white,https://images.craigslist.org/00i0i_ChcqziILO2...,CLEAN TITLE NO ACCEDINTS 3 YEARS WARRANTY D...,ca
freq,1,4501,2992,92601,11341,140663,111940,441688,487040,457659,114,170482,90481,93225,89322,167,160,52780


#### Missing Values 

Ways to handle null values:
1. Remove row(s) or column(s) with missing values from dataset
2. Fill is missing values with unharmful values

In [31]:
# Missing values for each variable
cars.isnull().sum().sort_values()

id                   0
url                  0
region               0
region_url           0
price                0
state                0
description          0
image_url            0
year              1513
title_status      3048
transmission      3705
fuel              3971
model             7975
long             10278
lat              10278
manufacturer     22749
odometer         92310
type            141516
drive           144129
paint_color     164692
cylinders       199669
vin             207409
condition       231919
size            341987
dtype: int64

In [21]:
# Percentage of missing values for each variable
round((cars.isnull().sum() / len(cars)) * 100, 4).sort_values()

id               0.0000
url              0.0000
region           0.0000
region_url       0.0000
price            0.0000
year             0.0000
state            0.0000
image_url        0.0027
description      0.0031
title_status     0.6009
transmission     0.7298
fuel             0.7820
model            1.5678
long             2.0197
lat              2.0197
manufacturer     4.4672
odometer        18.1178
type            27.7742
drive           28.2868
paint_color     32.3221
cylinders       39.1860
vin             40.7053
condition       45.5150
size            67.1151
dtype: float64

In [30]:
# Drop county
cars = cars.drop(['county', 'vin'], axis=1)

# Drop rows
cars = cars.dropna(subset=['image_url', 'description'])

In [55]:
# Print unique values, number of unique values and value frequency details
def variable_details(column):
    print('The are {} unique values in {}\n'.format(cars[column].nunique(), column))
    print('The unique values are \n{}\n'.format(cars[column].unique()))
    print('The value frequency is \n{}'.format(cars[column].value_counts()))

In [13]:
cars['year'].describe()

count    508050.000000
mean       2009.662236
std           8.567953
min        1900.000000
25%        2007.000000
50%        2011.000000
75%        2015.000000
max        2021.000000
Name: year, dtype: float64

In [14]:
# Populate missing values using mean
cars['year'] = cars['year'].fillna(cars['year'].mean())

In [56]:
variable_details('title_status')

The are 6 unique values in title_status

The unique values are 
['clean' 'rebuilt' 'salvage' 'lien' nan 'missing' 'parts only']

The value frequency is 
clean         487038
rebuilt        10473
salvage         5324
lien            2848
missing          607
parts only       223
Name: title_status, dtype: int64


In [22]:
# Populate missing values using 'missing' categorical value
cars['title_status'] = cars['title_status'].fillna('missing')

In [57]:
variable_details('transmission')

The are 3 unique values in transmission

The unique values are 
['manual' 'automatic' 'other']

The value frequency is 
automatic    457657
manual        33962
other         17942
Name: transmission, dtype: int64


In [46]:
# Populate missing values using 'other' categorical value
cars.transmission = cars.transmission.fillna('other')

In [58]:
variable_details('fuel')

The are 5 unique values in fuel

The unique values are 
['gas' 'diesel' 'other' 'electric' 'hybrid']

The value frequency is 
gas         441686
diesel       41986
other        21071
hybrid        4052
electric       766
Name: fuel, dtype: int64


In [49]:
# Populate missing values using 'other' categorical value
cars['fuel'].fillna('other', inplace=True)

In [69]:
variable_details('model')

The are 35851 unique values in model

The unique values are 
['golf r' 'f-150' 'sierra 1500' ... 'Camaro 2-door coupe'
 'Isuzu VehiCROSS' 'peterbilt 378']

The value frequency is 
f-150                    11341
silverado 1500            7405
1500                      7367
silverado                 5256
2500                      4439
                         ...  
new gti                      1
1500 sierra sle              1
x-runner                     1
traverse high country        1
ats 2.5l                     1
Name: model, Length: 35851, dtype: int64


In [70]:
variable_details('long')

The are 51467 unique values in long

The unique values are 
[-111.858 -111.884 -104.694 ... -115.226 -111.625 -111.97 ]

The value frequency is 
-84.1122     2339
-117.2370    2146
-74.2817     2077
-119.1280    2057
-84.4454     1813
             ... 
-149.8740       1
-85.3480        1
-87.2190        1
-121.5470       1
-78.9648        1
Name: long, Length: 51467, dtype: int64


In [71]:
variable_details('lat')

The are 51488 unique values in lat

The unique values are 
[40.7372 40.5881 40.3744 ... 40.758  38.8787 41.3112]

The value frequency is 
43.1824    2339
47.6561    2146
40.4688    2077
46.2348    2057
33.7865    1814
           ... 
39.1549       1
33.5253       1
33.9747       1
40.8451       1
27.3230       1
Name: lat, Length: 51488, dtype: int64


In [59]:
variable_details('manufacturer')

The are 43 unique values in manufacturer

The unique values are 
['volkswagen' 'ford' 'gmc' 'ram' 'toyota' 'chevrolet' 'mercedes-benz'
 'jeep' nan 'honda' 'lexus' 'cadillac' 'dodge' 'hyundai' 'subaru' 'bmw'
 'chrysler' 'nissan' 'mitsubishi' 'buick' 'acura' 'kia' 'rover'
 'alfa-romeo' 'mazda' 'saturn' 'pontiac' 'volvo' 'jaguar' 'fiat' 'audi'
 'mini' 'infiniti' 'lincoln' 'mercury' 'tesla' 'harley-davidson' 'porche'
 'land rover' 'aston-martin' 'ferrari' 'datsun' 'hennessey' 'morgan']

The value frequency is 
ford               92600
chevrolet          73661
toyota             38159
nissan             26435
ram                26177
honda              25585
jeep               23951
gmc                23126
dodge              19344
bmw                13468
hyundai            12182
mercedes-benz      11569
subaru             10259
volkswagen         10214
chrysler            8866
kia                 8821
cadillac            7304
buick               6357
lexus               6292
mazda        

In [60]:
variable_details('odometer')

The are 119873 unique values in odometer

The unique values are 
[6.3500e+04 1.0000e+01 7.5540e+03 ... 3.9457e+04 8.3200e+05 6.9730e+03]

The value frequency is 
0.0         2268
150000.0    1161
140000.0    1138
130000.0    1115
160000.0    1059
            ... 
45894.0        1
91786.0        1
183571.0       1
121871.0       1
71542.0        1
Name: odometer, Length: 119873, dtype: int64


In [61]:
variable_details('type')

The are 13 unique values in type

The unique values are 
['hatchback' nan 'pickup' 'SUV' 'sedan' 'truck' 'wagon' 'van' 'coupe'
 'convertible' 'other' 'offroad' 'mini-van' 'bus']

The value frequency is 
sedan          93225
SUV            92300
pickup         49547
truck          49509
coupe          19106
other          13631
hatchback      12850
wagon          10937
van            10579
convertible     7737
mini-van        7170
offroad          742
bus              712
Name: type, dtype: int64


In [72]:
# Populate missing values using 'other' categorical value
cars['type'] = cars['type'].fillna('other')

In [62]:
variable_details('drive')

The are 3 unique values in drive

The unique values are 
['4wd' nan 'fwd' 'rwd']

The value frequency is 
4wd    170480
fwd    127190
rwd     67762
Name: drive, dtype: int64


In [73]:
# Populate missing values using 'missing'
cars['drive'] = cars['drive'].fillna('missing')

In [63]:
variable_details('paint_color')

The are 12 unique values in paint_color

The unique values are 
['black' nan 'white' 'silver' 'brown' 'blue' 'grey' 'red' 'custom'
 'purple' 'yellow' 'green' 'orange']

The value frequency is 
white     89321
black     70580
silver    51352
blue      34584
red       34410
grey      33289
green      9192
custom     8894
brown      7758
yellow     2533
orange     2125
purple      831
Name: paint_color, dtype: int64


In [ ]:
# Populate missing values using 'other'
cars['paint_color'] = cars['paint_color'].fillna('other')

In [65]:
variable_details('cylinders')

The are 8 unique values in cylinders

The unique values are 
['4 cylinders' nan '8 cylinders' '6 cylinders' '10 cylinders'
 '5 cylinders' '3 cylinders' '12 cylinders' 'other']

The value frequency is 
6 cylinders     111940
8 cylinders      95370
4 cylinders      95308
5 cylinders       2714
10 cylinders      2299
other             1506
3 cylinders        562
12 cylinders       193
Name: cylinders, dtype: int64


In [ ]:
# Populate missing values using 'other' categorical value
cars['cylinders'] = cars['cylinders'].fillna('other')

In [67]:
variable_details('condition')

The are 6 unique values in condition

The unique values are 
['excellent' nan 'good' 'like new' 'fair' 'new' 'salvage']

The value frequency is 
excellent    140662
good          97261
like new      28166
fair           9461
new            1406
salvage         686
Name: condition, dtype: int64


In [68]:
variable_details('size')

The are 4 unique values in size

The unique values are 
['compact' nan 'mid-size' 'full-size' 'sub-compact']

The value frequency is 
full-size      90481
mid-size       47756
compact        25221
sub-compact     4116
Name: size, dtype: int64
